In [22]:
import pandas as pd
import numpy as np
import sys
import os
import requests

In [8]:
plop = pd.read_csv('./summary_file.csv')

In [18]:
vote_ids = plop.set_index('Unnamed: 0').T.reset_index()['vote_id']

# Vote retrieval

In [88]:
def get_vote_perc_per_x(df: pd.DataFrame, discri_col: str="member.group.label"):
    
    pos_vote_result = df[df["position"] == "VotePosition.FOR"]
    
    return (pos_vote_result.groupby(discri_col)['position'].count() / df.groupby(discri_col)['position'].count()).replace(np.nan, 0)

In [89]:
def get_vote_perc(df: pd.DataFrame):
    
    pos_vote_result = df[df["position"] == "VotePosition.FOR"]
    
    return (pos_vote_result['position'].count() / df['position'].count())


In [90]:
vote_result_tot = dict()
vote_result_per_group_dict = dict()

filters = {
    "member.country.code" : ["FRA"]
}

for vote_id in vote_ids:
    if not os.path.isfile(f'./{vote_id}.csv'):
        r = requests.get(f"https://howtheyvote.eu/api/votes/{vote_id}.csv")
        if r.ok:
#             print(f"saving to {os.path.abspath('./' + vote_id + '.csv')}")
            with open(f'./{vote_id}.csv', 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024 * 8): # magic 
                    if chunk:
                        f.write(chunk)
                        f.flush()
                        os.fsync(f.fileno())
        else:
            print("Download failed: status code {}\n{}".format(r.status_code, r.text))
    
    
    vote_data = pd.read_csv(f'./{vote_id}.csv')
    for col_name, col_values in filters.items():
        vote_data = vote_data[vote_data[col_name].isin(col_values)]
        
    vote_result_tot[vote_id] = get_vote_perc(vote_data)
    vote_result_per_group_dict[vote_id] = get_vote_perc_per_x(vote_data, "member.group.label")

Idea is to get a PCA association, as shown in this https://stats.stackexchange.com/questions/119746/what-is-the-proper-association-measure-of-a-variable-with-a-pca-component-on-a

Observation that Reconquete is not voting often

In [82]:
len(pd.concat(vote_dict, axis=1).T['European Conservatives and Reformists'].dropna()) / len(pd.concat(vote_dict, axis=1).T['European Conservatives and Reformists'])

0.04180064308681672

# Vote vectorisation + Variance metric

In [110]:
vote_df = pd.concat(vote_dict, axis=1)

In [111]:
variance = vote_df.std(axis=0)

count    311.000000
mean       0.365538
std        0.162650
min        0.000000
25%        0.369646
50%        0.408248
75%        0.480181
max        0.547723
dtype: float64

In [109]:
vote_tot_df = pd.DataFrame([vote_result_tot]).T
non_full_aggreement = vote_tot_df[vote_tot_df[0] < 0.98].index

Index(['117281', '134945', '162202', '147523', '156848', '122259', '163061',
       '133727', '116296', '153640',
       ...
       '166566', '134333', '147458', '134875', '117284', '134950', '146836',
       '133938', '126257', '156755'],
      dtype='object', length=281)

In [121]:
vote_df.cov().abs().mean(axis=1).nsmallest(60)

126265    0.000000
128845    0.000000
126264    0.000000
128854    0.000000
117268    0.000000
116785    0.000000
113694    0.000000
125365    0.000000
126267    0.000000
131706    0.000000
115402    0.000000
130717    0.000000
116291    0.000000
128853    0.000000
126266    0.000000
130716    0.000000
137603    0.000000
139917    0.000000
124725    0.001807
124724    0.001807
124723    0.001807
134350    0.002884
125374    0.003274
130599    0.003274
127024    0.004134
127026    0.004134
134874    0.004134
134873    0.004134
138170    0.004187
127180    0.005457
111546    0.006597
112552    0.006616
118524    0.007034
128852    0.007126
126262    0.007285
150069    0.008256
133940    0.008267
127831    0.008267
127627    0.008511
127624    0.008511
153640    0.009191
146606    0.009333
160132    0.010102
162987    0.010277
163095    0.011055
155525    0.011643
155882    0.012588
148324    0.013809
131708    0.014252
151561    0.016230
112530    0.016996
146358    0.017230
146315    0.